<a href="https://colab.research.google.com/github/gchacko/1.1.1.1/blob/master/two_python_executors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langgraph>=0.0.18 langchain-openai>=0.0.5 openai>=1.12.0 python-dotenv>=0.21.0 langchain-anthropic>=0.1.1 python-dotenv==1.0.1


In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from typing import TypedDict,Dict,  List, Any
from langgraph.graph import StateGraph, END, START
from langchain_core.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
import ast
import json
import tempfile
from pathlib import Path
import subprocess

class GraphState(TypedDict):
    error: str
    messages: List
    generation: str
    iterations: int
    execution_method: str

def validate_code(code_str: str) -> bool:
    """Basic validation of code string"""
    try:
        ast.parse(code_str)
        return True
    except SyntaxError:
        return False

def execute_with_exec(code_str: str, test_data: Dict[str, Any]) -> Dict[str, Any]:
    """Execute Python code string using exec()"""
    if not validate_code(code_str):
        raise ValueError("Invalid code")
    try:
        namespace = test_data
        exec(f"result = {code_str}", namespace)
        return {'result': namespace['result']}
    except Exception as e:
        return {'error': str(e)}


def execute_with_subprocess(code_str: str, test_data: Dict[str, Any]) -> Dict[str, Any]:
    """Execute Python code using subprocess.run()"""
    if not validate_code(code_str):
        return {'error': 'Invalid code syntax'}

    with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as tmp:
        script_content = (
            f"import json\n"
            f"import sys\n"
            f"input_data = json.loads(sys.argv[1])\n"
            f"numbers = input_data['numbers']\n"
            f"result = {code_str}\n"
            f"print(json.dumps({{'result': result}}))"
        )
        tmp.write(script_content)
        script_path = tmp.name

    try:
        input_json = json.dumps(test_data)
        result = subprocess.run(
            ['python', script_path, input_json],
            capture_output=True,
            text=True,
            check=True
        )
        return json.loads(result.stdout)
    except subprocess.CalledProcessError as e:
        return {'error': f"Process error: {e.stderr}"}
    except json.JSONDecodeError:
        return {'error': "Failed to parse output"}
    finally:
        Path(script_path).unlink()

def generate(state: GraphState) -> GraphState:
    """Generate a code solution"""
    messages = state["messages"]
    error = state["error"]
    iterations = state["iterations"]

    code_gen_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a coding assistant. Answer the user question by providing
         executable Python code. The code MUST be a complete expression, for example:
         'sum(x**2 for x in numbers)'. Don't include variable initializations or test code,
         just the core expression. If there were previous errors, fix them."""),
        ("human", "{question}"),
        ("human", "Previous errors if any: {error}")
    ])

    llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)
    generation = llm.invoke(code_gen_prompt.format_messages(
        question=messages[0][1],
        error=error
    ))

    print(f"Attempt #{iterations + 1}")

    return {
        "generation": generation.content,
        "messages": messages,
        "iterations": iterations + 1,
        "error": "",
        "execution_method": state["execution_method"]
    }

def code_check(state: GraphState) -> GraphState:
    """Check code using specified execution method"""
    code_solution = state["generation"]
    iterations = state["iterations"]
    test_data = {'numbers': [1, 2, 3]}

    execution_method = (execute_with_subprocess if state["execution_method"] == "subprocess"
                       else execute_with_exec)

    try:
        result = execution_method(code_solution, test_data)
        if 'error' in result:
            print(f"Failed attempt #{iterations} with error: {result['error']}")
            return {**state, "error": result['error']}

        print(f"Success on attempt #{iterations}")
        return {**state, "error": "no"}
    except Exception as e:
        print(f"Failed attempt #{iterations} with error: {str(e)}")
        return {**state, "error": str(e)}

def should_retry(state: GraphState) -> str:
    """Determine if we should retry on error"""
    error = state["error"]
    iterations = state["iterations"]

    if error == "no":
        return "end"
    if iterations < 3:
        print(f"Retrying after attempt #{iterations}")
        return "generate"
    print(f"Giving up after {iterations} attempts")
    return "end"

# Build graph
workflow = StateGraph(GraphState)
workflow.add_node("generate", generate)
workflow.add_node("check_code", code_check)

# Connect nodes
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "check_code")

workflow.add_conditional_edges(
    "check_code",
    should_retry,
    {
        "generate": "generate",
        "end": END
    }
)

app = workflow.compile()

## Getting Started with exec() and subprocess.run()

In [ ]:
code = """
def greet(name):
    return f"Hello, {name}!"

message = greet("Alice")
"""

exec(code)

print(message)

In [ ]:
import subprocess

result = subprocess.run(
    ["python", "-c", "print('Hello, World!')"],
    capture_output=True,
    text=True
)

In [ ]:
result.stdout.strip()

## Executing the Agent Code

In [ ]:
from agent import app, execute_with_exec, execute_with_subprocess

# Test questions
questions = [
    "Write code that takes a list of numbers and returns their sum",
    "Write code that takes a list of numbers and returns the sum of their squares",
    "Write code that takes a list of numbers and returns the sum of the squares of even numbers that are greater than 3"
]

test_data = {'numbers': [1, 2, 3, 4, 5, 6, 7, 8]}

In [ ]:
from IPython.display import display, Markdown

def test_questions_exec(questions,test_data):
    for i, question in enumerate(questions, 1):
        result = app.invoke({
            "messages": [("user", question)],
            "iterations": 0,
            "error": "",
            "generation": None,
            "execution_method": "exec"
        })

        output = execute_with_exec(result["generation"], test_data)

        display(Markdown(f"""
## Question {i} with exec()
{question}

### Solution (after {result['iterations']} iterations):
```python
{result["generation"]}
```

### Test Output ([1,2,3,4,5,6,7,8]):
```
{output}
```
---
"""))

In [ ]:
test_questions_exec(questions,test_data)

In [ ]:

def test_questions_subprocess(questions):
    for i, question in enumerate(questions, 1):
        result = app.invoke({
            "messages": [("user", question)],
            "iterations": 0,
            "error": "",
            "generation": None,
            "execution_method": "subprocess"
        })

        test_data = {'numbers': [1, 2, 3, 4, 5, 6, 7, 8]}
        output = execute_with_subprocess(result["generation"], test_data)

        display(Markdown(f"""
## Question {i} with subprocess
{question}

### Solution (after {result['iterations']} iterations):
```python
{result["generation"]}
```

### Test Output ([1,2,3,4,5,6,7,8]):
```
{output}
```
---
"""))

In [ ]:
test_questions_subprocess(questions)